In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.covariance import EllipticEnvelope


In [2]:
df = pd.read_csv('dataset/Dortmund_features.csv')

print("Shape of raw data:", df.shape)
print("Columns:", df.columns.tolist())

sub_id_col = 'Unnamed: 0'

eeg_features = [c for c in df.columns if c != sub_id_col]
print("Number of EEG features:", len(eeg_features))


Shape of raw data: (600, 148)
Columns: ['Unnamed: 0', 'nl_rqa mean diag length', 'kurtosis_amp_theta', 'lt_delta', 'envelope_mean_alpha', 'nl_correlation dim', 'kurtosis_amp_alpha', 'mod_index_theta-gamma', 'nl_rqa rte', 'nl_higuchi fractal dim', 'acw50', 'envelope_std_beta', 'wt_delta', 'mod_index_theta-beta', 'spec_ent_beta', 'wt_beta', 'spec_ent_delta', 'nl_lyapunov exponent', 'nl_katz fractal dim', 'mod_index_theta-alpha', 'total_power_theta', 'skewness_amp_theta', 'mod_index_alpha-beta', 'nl_rqa determinism', 'rel_amp_gamma', 'skewness_amp_gamma', 'spec_ent_gamma', 'nl_rqa laminarity', 'acw0', 'envelope_mean_beta', 'total_power_beta', 'kurtosis_amp_beta', 'spec_ent_theta', 'kurtosis_amp_gamma', 'wt_alpha', 'spec_ent_alpha', 'wt_theta', 'rel_amp_theta', 'envelope_std_theta', 'rel_amp_beta', 'total_power_alpha', 'skewness_amp_alpha', 'skewness_amp_beta', 'nl_hjorth mobility', 'lt_alpha', 'skewness_amp_delta', 'envelope_std_gamma', 'wt_gamma', 'lt_theta', 'nl_rqa trapping time', 'kur

In [ ]:
# histogram for all features
for col in eeg_features:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(col)
    plt.tight_layout()
    plt.show()